In [1]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

import albumentations as A
from albumentations.pytorch import ToTensorV2

import pandas as pd
import h5py
from tqdm import tqdm
from datasets import load_dataset
from transformers import AutoImageProcessor, ResNetForImageClassification



In [2]:
# CONFIG: All important training settings
CONFIG = {
    # Random seed — makes results reproducible. Change it to get different shuffles.
    "seed": 42,
    
    # Number of images to use from the training and validation sets.
    # Good for testing smaller runs before full-scale training.
    "n_samples_train": 3000,
    "n_samples_val": 3000,
    
    # Total number of passes (epochs) through the entire training data.
    "epochs": 50,
    
    # Target size to which all images will be resized (square images here).
    "img_size": 384,
    
    # Model architecture to use — EfficientNet B0 variant from the timm library.
    "model_name": "tf_efficientnet_b0_ns",
    
    # Path to pretrained weights for this model (speeds up convergence).
    "checkpoint_path": "/kaggle/input/tf-efficientnet/pytorch/tf-efficientnet-b0/1/tf_efficientnet_b0_aa-827b6e33.pth",
    
    # How many images to process at once during training and validation.
    "train_batch_size": 200,
    "valid_batch_size": 64,
    
    # Learning rate — how big each weight update step should be.
    "learning_rate": 1e-4,
    
    # Scheduler type to adjust learning rate over time.
    # 'CosineAnnealingLR' smoothly decreases LR following a cosine curve.
    "scheduler": 'CosineAnnealingLR',
    
    # The lowest learning rate allowed by the scheduler.
    "min_lr": 1e-6,
    
    # Number of iterations before cosine annealing restarts.
    "T_max": 500,
    
    # Weight decay — a small penalty on large weights to reduce overfitting.
    "weight_decay": 1e-6,
    
    # Which fold of K-Fold Cross Validation to train on right now.
    "fold": 4,
    # Total number of folds for cross-validation.
    "n_fold": 5,
    
    # Gradient accumulation steps — useful when batches are too big for GPU.
    "n_accumulate": 1,
    
    # Use GPU if available, otherwise fall back to CPU.
    "device": torch.device("cuda:0" if torch.cuda.is_available() else "cpu"),
}


In [3]:
# Main dataset folder
ROOT_DIR = "/home/mccruz/isic/ISIC2024_Skin_Cancer_Detection"

# File paths for metadata, images, and sample submission
TEST_CSV = f"{ROOT_DIR}/train-metadata.csv"
TEST_HDF = f"{ROOT_DIR}/train-image.hdf5"
SAMPLE = f"{ROOT_DIR}/sample_submission.csv"

# Best model weights to load
BEST_WEIGHT = f"{ROOT_DIR}/AUROC0.4996_Loss0.1373_epoch1.pth"


setting the folder and file paths so my code knows where to find the dataset and the trained model weights.

In [4]:
# Dataset folder (Windows)
ROOT_DIR = r"C:\Users\Yashwanth\isic"

# Files used by the notebook
TEST_CSV = f"{ROOT_DIR}\\train-metadata.csv"
TEST_HDF = f"{ROOT_DIR}\\train-image.hdf5"
SAMPLE   = f"{ROOT_DIR}\\sample_submission.csv"

# Best model weights (change name if yours differs)
BEST_WEIGHT = f"{ROOT_DIR}\\AUROC0.4996_Loss0.1373_epoch1.pth"


In [5]:
df = pd.read_csv(TEST_CSV)
df["target"] = 0
df


C:\Users\Yashwanth\AppData\Local\Temp\ipykernel_13480\1055831860.py:1: DtypeWarning: Columns (51,52) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(TEST_CSV)


,isic_id,target,patient_id,age_approx,sex,anatom_site_general,clin_size_long_diam_mm,image_type,tbp_tile_type,tbp_lv_A,...,lesion_id,iddx_full,iddx_1,iddx_2,iddx_3,iddx_4,iddx_5,mel_mitotic_index,mel_thick_mm,tbp_lv_dnn_lesion_confidence
0,ISIC_0015670,0,IP_1235828,60.0,male,lower extremity,3.04,TBP tile: close-up,3D: white,20.244422,...,NaN,Benign,Benign,NaN,NaN,NaN,NaN,NaN,NaN,97.517282
1,ISIC_0015845,0,IP_8170065,60.0,male,head/neck,1.10,TBP tile: close-up,3D: white,31.712570,...,IL_6727506,Benign,Benign,NaN,NaN,NaN,NaN,NaN,NaN,3.141455
2,ISIC_0015864,0,IP_6724798,60.0,male,posterior torso,3.40,TBP tile: close-up,3D: XP,22.575830,...,NaN,Benign,Benign,NaN,NaN,NaN,NaN,NaN,NaN,99.804040
3,ISIC_0015902,0,IP_4111386,65.0,male,anterior torso,3.22,TBP tile: close-up,3D: XP,14.242329,...,NaN,Benign,Benign,NaN,NaN,NaN,NaN,NaN,NaN,99.989998
4,ISIC_0024200,0,IP_8313778,55.0,male,anterior torso,2.73,TBP tile: close-up,3D: white,24.725520,...,NaN,Benign,Benign,NaN,NaN,NaN,NaN,NaN,NaN,70.442510
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
401054,ISIC_9999937,0,IP_1140263,70.0,male,anterior torso,6.80,TBP tile: close-up,3D: XP,22.574335,...,IL_9520694,Benign,Benign,NaN,NaN,NaN,NaN,NaN,NaN,99.999988
401055,ISIC_9999951,0,IP_5678181,60.0,male,posterior torso,3.11,TBP tile: close-up,3D: white,19.977640,...,NaN,Benign,Benign,NaN,NaN,NaN,NaN,NaN,NaN,99.999820
401056,ISIC_9999960,0,IP_0076153,65.0,female,anterior torso,2.05,TBP tile: close-up,3D: XP,17.332567,...,IL_9852274,Benign,Benign,NaN,NaN,NaN,NaN,NaN,NaN,99.999416
401057,ISIC_9999964,0,IP_5231513,30.0,female,anterior torso,2.80,TBP tile: close-up,3D: XP,22.288570,...,NaN,Benign,Benign,NaN,NaN,NaN,NaN,NaN,NaN,100.000000


In [6]:
# Load sample submission file
df_sub = pd.read_csv(SAMPLE)
df_sub

,isic_id,target
0,ISIC_0015657,0.3
1,ISIC_0015729,0.3
2,ISIC_0015740,0.3


In [ ]:
import os

# Copy training table
df = train_df.copy()

# List all training image paths
train_images = sorted(glob.glob(f"{TRAIN_DIR}/*.jpg"))

# Keep only the filename part (e.g., "abc123.jpg")
train_files = set(os.path.basename(p) for p in train_images)

# Build expected filename from isic_id (e.g., "abc123.jpg")
df["file_name"] = df["isic_id"].astype(str) + ".jpg"

# Keep rows whose filename exists in the folder
df = df[df["file_name"].isin(train_files)].reset_index(drop=True)

# Build full path using TRAIN_DIR + filename
df["image_path"] = df["file_name"].apply(lambda x: os.path.join(TRAIN_DIR, x))

# Print counts: images, positives, negatives, unique patients (shape-style)
print("# of images , # of positive cases, # of negative cases, # of patients")
print(df.shape, df["target"].sum(), (df["target"] == 0).sum(), df["patient_id"].unique().shape)

# Split into positive/negative subsets
df_positive = df[df["target"] == 1].reset_index(drop=True)
df_negative = df[df["target"] == 0].reset_index(drop=True)


# of images , # of positive cases, # of negative cases, # of patients
(0, 57) 0 0 (0,)


### What this output means

| isic_id        | target |
|----------------|--------|
| ISIC_0015657   | 0.3    |
| ISIC_0015729   | 0.3    |
| ISIC_0015740   | 0.3    |

- **isic_id** → Unique ID for each skin lesion image.
- **target** → Model's predicted probability (0 to 1) that the lesion is malignant.
- **Closer to 1** → more likely malignant.
- **Closer to 0** → more likely benign.
- Usually, **0.5** is used as the cut-off:
  - ≥ 0.5 → malignant  
  - < 0.5 → benign  

Example: All above are `0.3` → **30% chance malignant** → predicted **benign**.


In [7]:
TRAIN_DIR = f'{ROOT_DIR}/train-image/image'

In [8]:
# Get full path to a training image by its ID
def get_train_file_path(image_id):
    return f"{TRAIN_DIR}/{image_id}.jpg"

# Display an image from a given file path
def show_im(image_id):
    image = mpimg.imread(image_id)  # read image
    plt.imshow(image)               # show image
    plt.show()

In [9]:
# Create a new column with the full image file path
df['image_path'] = df['isic_id'].apply(get_train_file_path)

In [10]:
# Custom PyTorch dataset for ISIC images stored in an HDF5 file
class ISICDataset(Dataset):
    def __init__(self, df, file_hdf, transforms=None):
        self.df = df
        self.fp_hdf = h5py.File(file_hdf, mode="r")
        self.isic_ids = df['isic_id'].values
        self.targets = df['target'].values
        self.transforms = transforms
        
    def __len__(self):
        return len(self.isic_ids)
    
    def __getitem__(self, index):
        isic_id = self.isic_ids[index]
        img = np.array( Image.open(BytesIO(self.fp_hdf[isic_id][()])) )
        target = self.targets[index]
        
        if self.transforms:
            img = self.transforms(image=img)["image"]
            
        return {
            'image': img,
            'target': target,
        }

In [11]:
data_transforms = {
    "valid": A.Compose([
        A.Resize(CONFIG['img_size'], CONFIG['img_size']),
        A.Normalize(
                mean=[0.485, 0.456, 0.406], 
                std=[0.229, 0.224, 0.225], 
                max_pixel_value=255.0, 
                p=1.0
            ),
        ToTensorV2()], p=1.)
}

In [12]:

# Device (optional)
device = "cuda" if torch.cuda.is_available() else "cpu"

# --- Demo using a sample image from the datasets hub ---
dataset = load_dataset("huggingface/cats-image")
image = dataset["test"]["image"][0]  # a PIL Image

processor = AutoImageProcessor.from_pretrained("microsoft/resnet-50")
model = ResNetForImageClassification.from_pretrained("microsoft/resnet-50").to(device)
model.eval()

inputs = processor(image, return_tensors="pt").to(device)

with torch.no_grad():
    logits = model(**inputs).logits

pred_id = int(logits.argmax(-1).item())
print(model.config.id2label[pred_id])

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


tiger cat


In [13]:
new_classifier = nn.Sequential(
    nn.Flatten(start_dim=1, end_dim=-1),
    nn.Linear(in_features=2048, out_features=2, bias=True)
)

# Replace the old classifier with the new one
model.classifier = new_classifier

for param in model.parameters():
    param.requires_grad = False

    
for param in model.classifier.parameters():
    param.requires_grad = True

# Verify which parameters require gradients
#for name, param in model.named_parameters():
#    print(name, param.requires_grad)

In [14]:
test_dataset = ISICDataset(df, TEST_HDF, transforms=data_transforms["valid"])
test_loader = DataLoader(test_dataset, batch_size=CONFIG['valid_batch_size'], 
                          num_workers=2, shuffle=False, pin_memory=True)

In [ ]:
import torch
import numpy as np
from tqdm import tqdm

# Use GPU if available, otherwise CPU
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device).eval()

# Make DataLoader faster
test_loader = DataLoader(
    test_dataset,
    batch_size=32,              # process 32 images at once
    shuffle=False,
    num_workers=4,              # load data in 4 parallel workers
    pin_memory=True             # speedup for GPU
)

# Runing inference
preds = []
torch.backends.cudnn.benchmark = True  # extra speed on GPU
with torch.inference_mode():           # faster than no_grad()
    for _, data in tqdm(enumerate(test_loader), total=len(test_loader)):
        images = data['image'].to(device, dtype=torch.float, non_blocking=True)
        outputs = model(images)
        preds.append(outputs.cpu().numpy())

preds = np.concatenate(preds).ravel()

print(" Inference complete! Predictions shape:", preds.shape)


C:\Users\Yashwanth\AppData\Roaming\Python\Python313\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


In [ ]:
preds = []
with torch.no_grad():
    bar = tqdm(enumerate(test_loader), total=len(test_loader))
    for step, data in bar:        
        images = data['image'].to(CONFIG["device"], dtype=torch.float)        
        batch_size = images.size(0)
        outputs = model(images)
        preds.append( outputs.detach().cpu().numpy() )
preds = np.concatenate(preds).flatten()

C:\Users\Yashwanth\AppData\Roaming\Python\Python313\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
